#### Install the useful packages.

In [1]:
myPackages <- c("easypackages")
#the argument `lib.loc = "~/rlib"` is used because the packages were installed in this folder in my case
lapply(myPackages, library, lib.loc = "~/rlib", character.only = TRUE) 
libraries("dplyr", "tidyr")

[[1]]
[1] "easypackages" "stats"        "graphics"     "grDevices"    "utils"       
[6] "datasets"     "methods"      "base"

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

All packages loaded successfully



#### Import the gallup data that contain every country.

In [2]:
gallupRaw <- rio::import("~/gallup data/The_Gallup_042722.dat")

Ambiguous file format ('.dat'), but attempting 'data.table::fread("~/gallup data/The_Gallup_042722.dat")'



We need country name, country code, weight, survey date, survey year, ls, hope, PA, NA

Filter the data and save the it for future use.

In [6]:
gallupExtracted <- gallupRaw %>% select(COUNTRYNEW, COUNTRY_ISO3, WGT, 
                                        FIELD_DATE, WP4.INTERVIEWDATE = WP4, YEAR_WAVE, 
                                        YEAR_CALENDAR, ls = WP16, hope = WP18, 
                                        smile = WP63, enjoy = WP67, worry = WP69, 
                                        sad = WP70, stress = WP71, anger = WP74,
                                        WP1219.MALE = WP1219, WP1220.AGE = WP1220, EMP_2010,
                                        WP1223.MARITAL = WP1223)
write.csv(gallupExtracted, "gallupExtracted.csv", row.names = FALSE)

In [7]:
gallupExtracted <- rio::import("gallupExtracted.csv")
colnames(gallupExtracted)
head(gallupExtracted)
dim(gallupExtracted)

[1] "COUNTRYNEW"        "COUNTRY_ISO3"      "WGT"              
 [4] "FIELD_DATE"        "WP4.INTERVIEWDATE" "YEAR_WAVE"        
 [7] "YEAR_CALENDAR"     "ls"                "hope"             
[10] "smile"             "enjoy"             "worry"            
[13] "sad"               "stress"            "anger"            
[16] "WP1219.MALE"       "WP1220.AGE"        "EMP_2010"         
[19] "WP1223.MARITAL"

,COUNTRYNEW,COUNTRY_ISO3,WGT,FIELD_DATE,WP4.INTERVIEWDATE,YEAR_WAVE,YEAR_CALENDAR,ls,hope,smile,enjoy,worry,sad,stress,anger,WP1219.MALE,WP1220.AGE,EMP_2010,WP1223.MARITAL
,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>
1,United States,USA,0.6689163,7/1/2006,,2006,2006,6,8,1,1,2,2,2,2,2,47,NA,2
2,United States,USA,0.1966555,7/1/2006,,2006,2006,8,98,1,1,2,2,2,2,2,67,NA,5
3,United States,USA,1.3032635,7/1/2006,,2006,2006,8,8,1,1,1,2,1,2,1,62,NA,2
4,United States,USA,0.6359364,7/1/2006,,2006,2006,8,8,1,1,2,2,2,2,1,82,NA,5
5,United States,USA,0.7358092,7/1/2006,,2006,2006,10,10,1,1,2,2,2,2,2,62,NA,2
6,United States,USA,1.7369582,7/1/2006,,2006,2006,5,10,1,1,1,1,1,1,2,32,NA,2


[1] 2450403      19

Create a function for changing the "DK" / "Refused" response to NA.

In [5]:
na_codes <- function(x, ...) {
    for(y in list(...))
        x[x == y] <- NA
    x
}

Change the "DK" / "Refused" response to NA

Change WP1219.MALE from [1 Male, 2 Female] to [1 Yes, 0 No]

Change the responses to SWB from [1 Yes, 2 No] to [1 Yes, 0 No] for getting the means for individual positive affect and negative affect

In [9]:
gallupAssas <- gallupExtracted

gallupAssas$WP1220.AGE <-  na_codes(gallupAssas$WP1220.AGE, 100)
gallupAssas$WP1223.MARITAL <-  na_codes(gallupAssas$WP1223.MARITAL, 6,7)

#WP1219.MALE
gallupAssas$WP1219.MALE <- 2 - gallupAssas$WP1219.MALE

#convert empty WP4.INTERVIEWDATE to NA
gallupAssas <- gallupAssas %>% mutate(WP4.INTERVIEWDATE = na_if(WP4.INTERVIEWDATE, "")) 
#life satisfication
gallupAssas$ls <- na_codes(gallupAssas$ls, 98, 99)
#hope
gallupAssas$hope <- na_codes(gallupAssas$hope, 98, 99)
#pa and #na
gallupAssas$smile <- 2 - na_codes(gallupAssas$smile, 3, 4)
gallupAssas$enjoy <- 2 - na_codes(gallupAssas$enjoy, 3, 4)
gallupAssas$worry <- 2 - na_codes(gallupAssas$worry, 3, 4)
gallupAssas$sad <- 2 - na_codes(gallupAssas$sad, 3, 4)
gallupAssas$stress <- 2 - na_codes(gallupAssas$stress, 3, 4)
gallupAssas$anger <- 2 - na_codes(gallupAssas$anger, 3, 4)
gallupAssas <- gallupAssas %>% 
 rowwise() %>% 
 mutate(pa = mean(cbind(enjoy, smile), na.rm=TRUE),
        na = mean(cbind(worry, sad, stress, anger), na.rm=TRUE)) %>% ungroup
head(gallupAssas)

COUNTRYNEW,COUNTRY_ISO3,WGT,FIELD_DATE,WP4.INTERVIEWDATE,YEAR_WAVE,YEAR_CALENDAR,ls,hope,smile,⋯,worry,sad,stress,anger,WP1219.MALE,WP1220.AGE,EMP_2010,WP1223.MARITAL,pa,na
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>
United States,USA,0.6689163,7/1/2006,NA,2006,2006,6,8,1,⋯,0,0,0,0,0,47,NA,2,1,0.0
United States,USA,0.1966555,7/1/2006,NA,2006,2006,8,NA,1,⋯,0,0,0,0,0,67,NA,5,1,0.0
United States,USA,1.3032635,7/1/2006,NA,2006,2006,8,8,1,⋯,1,0,1,0,1,62,NA,2,1,0.5
United States,USA,0.6359364,7/1/2006,NA,2006,2006,8,8,1,⋯,0,0,0,0,1,82,NA,5,1,0.0
United States,USA,0.7358092,7/1/2006,NA,2006,2006,10,10,1,⋯,0,0,0,0,0,62,NA,2,1,0.0
United States,USA,1.7369582,7/1/2006,NA,2006,2006,5,10,1,⋯,1,1,1,1,0,32,NA,2,1,1.0


In [11]:
write.csv(gallupAssas, "gallupAssas.csv")

In [12]:
colnames(gallupAssas)
dim(gallupAssas)

[1] "COUNTRYNEW"        "COUNTRY_ISO3"      "WGT"              
 [4] "FIELD_DATE"        "WP4.INTERVIEWDATE" "YEAR_WAVE"        
 [7] "YEAR_CALENDAR"     "ls"                "hope"             
[10] "smile"             "enjoy"             "worry"            
[13] "sad"               "stress"            "anger"            
[16] "WP1219.MALE"       "WP1220.AGE"        "EMP_2010"         
[19] "WP1223.MARITAL"    "pa"                "na"

[1] 2450403      21